## PyTorch 선형 회귀 구현

In [37]:
import pandas as pd
import numpy as np 
import torch 

import matplotlib.pyplot as plt 
%matplotlib inline

In [38]:
torch.manual_seed(7777)

In [39]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['const'] = np.ones(df.shape[0])
df.tail()

/Users/newrandom/miniforge3/envs/TF/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Californ

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


In [40]:
df.shape

(506, 14)

In [41]:
# torch 연산 운용
x = torch.tensor(df.values)
y = torch.tensor(boston.target).view(-1, 1)

In [42]:
x.shape, y.shape

(torch.Size([506, 14]), torch.Size([506, 1]))

In [43]:
XT = torch.transpose(x, 0, 1)

In [44]:
XT.shape

torch.Size([14, 506])

In [45]:
w = torch.mm(torch.mm(torch.linalg.inv(torch.mm(XT, x)), XT),y)

In [46]:
y_pred = torch.matmul(x, w)
                # 예측치가 나온다.

In [47]:
y_pred[19]

tensor([18.4061], dtype=torch.float64)

In [48]:
boston.target[19]

18.2

#### Gradient descent 방식

In [49]:
w = torch.rand((14, 1), dtype = torch.float64, requires_grad =True) 
b = torch.rand((1,1), dtype = torch.float64, requires_grad =True)     # bias 는 1,1

In [50]:
z = x.mm(w) + b

In [51]:
loss = torch.mean((z-y)**2)

In [52]:
loss.backward()

In [53]:
w.grad, b.grad

(tensor([[4.6543e+03],
         [1.1592e+04],
         [1.2824e+04],
         [7.4684e+01],
         [6.1311e+02],
         [6.7807e+03],
         [7.6970e+04],
         [3.9565e+03],
         [1.1487e+04],
         [4.6771e+05],
         [2.0181e+04],
         [3.9043e+05],
         [1.4341e+04],
         [1.0850e+03]], dtype=torch.float64),
 tensor([[1084.9567]], dtype=torch.float64))

In [54]:
print(loss)

tensor(303557.1678, dtype=torch.float64, grad_fn=<MeanBackward0>)


>`with torch.no_grad()` / `.detach()` 를 써서 print 하기

In [55]:
print(loss.detach().numpy())

with torch.no_grad():
    print(loss.numpy())

303557.16782532405
303557.16782532405


In [56]:
loss.item()
        # 아주 간편하게 뽑을 수 있다.

303557.16782532405

In [57]:
w.data

tensor([[0.9625],
        [0.8798],
        [0.6916],
        [0.0976],
        [0.5441],
        [0.6069],
        [0.7656],
        [0.2674],
        [0.7172],
        [0.4656],
        [0.7843],
        [0.7482],
        [0.5919],
        [0.0070]], dtype=torch.float64)

#### assign 대신에 data에 접근해서 값을 수정 

tensor.data = 다른데이터

In [58]:
lr = 3e-7

In [59]:
for epoch in range(100):
    z = x.matmul(w) + b
    loss = torch.mean((y-z)**2)

    loss.backward()
            # backward를 하면 이전의 값이 축적이 되서 업데이트가 제대로 이뤄지지 않는다.

    w.data = w.data - w.grad * lr
    b.data = b.data - b.grad * lr
    
    print('{} - loss : {}'.format(epoch, loss.item()))

    w.grad.zero_()  # w 안의 grad가 0으로 초기화 된다.
    b.grad.zero_()

0 - loss : 303557.16782532405
1 - loss : 119223.98659357797
2 - loss : 79124.14612788572
3 - loss : 52663.83577694337
4 - loss : 35201.07677258205
5 - loss : 23673.78108793309
6 - loss : 16062.0358640329
7 - loss : 11033.399307362255
8 - loss : 7708.920110912625
9 - loss : 5508.792152027679
10 - loss : 4050.540603569035
11 - loss : 3081.8603939022373
12 - loss : 2436.3130901562427
13 - loss : 2004.1023413418766
14 - loss : 1712.795894144592
15 - loss : 1514.6088658572073
16 - loss : 1378.0151509354187
17 - loss : 1282.2138811644047
18 - loss : 1213.4792221507973
19 - loss : 1162.7525127944639
20 - loss : 1124.0539164537608
21 - loss : 1093.4346631281346
22 - loss : 1068.285892621278
23 - loss : 1046.8827296477364
24 - loss : 1028.0835297906765
25 - loss : 1011.1314840058294
26 - loss : 995.5237440302701
27 - loss : 980.925088042254
28 - loss : 967.11096738067
29 - loss : 953.929934560666
30 - loss : 941.2788562389089
31 - loss : 929.0865598461073
32 - loss : 917.3030435611212
33 - loss

In [60]:
for epoch in range(100):
    z = x.matmul(w) + b
    loss = torch.mean((y-z)**2)

    grads = torch.autograd.grad(loss, [w,b])

    w.data = w.data - grads[0] * lr
    b.data = b.data - grads[1] * lr
    
    print('{} - loss : {}'.format(epoch, loss.item()))

    w.grad.zero_()  # w 안의 grad가 0으로 초기화 된다.
    b.grad.zero_()

0 - loss : 565.7680959473693
1 - loss : 563.9983008315796
2 - loss : 562.273925345359
3 - loss : 560.5937013194101
4 - loss : 558.9563960453837
5 - loss : 557.3608112842763
6 - loss : 555.8057823025583
7 - loss : 554.290176935254
8 - loss : 552.8128946752206
9 - loss : 551.3728657878955
10 - loss : 549.9690504507961
11 - loss : 548.6004379170832
12 - loss : 547.2660457025132
13 - loss : 545.9649187951268
14 - loss : 544.6961288870359
15 - loss : 543.4587736276931
16 - loss : 542.2519758980418
17 - loss : 541.074883104964
18 - loss : 539.9266664954566
19 - loss : 538.8065204899867
20 - loss : 537.7136620344868
21 - loss : 536.647329970473
22 - loss : 535.6067844227732
23 - loss : 534.5913062043795
24 - loss : 533.6001962379406
25 - loss : 532.6327749934331
26 - loss : 531.6883819415555
27 - loss : 530.766375022408
28 - loss : 529.8661301290299
29 - loss : 528.9870406053783
30 - loss : 528.1285167583453
31 - loss : 527.2899853834208
32 - loss : 526.4708893036188
33 - loss : 525.670686921

#### optimizer 사용하기

In [61]:
opt = torch.optim.SGD([w, b], lr=lr)

In [62]:
for epoch in range(100):
    z = x.matmul(w) + b
    loss = torch.mean((z - y )**2)

    loss.backward()

    opt.step()
    print('{} - loss : {}'.format(epoch, loss.item()))
    opt.zero_grad()     # gradient 초기화

0 - loss : 496.7091936065956
1 - loss : 496.472825436027
2 - loss : 496.23918279497764
3 - loss : 496.00819126859074
4 - loss : 495.7797785218309
5 - loss : 495.55387424132726
6 - loss : 495.33041007884174
7 - loss : 495.10931959631796
8 - loss : 494.8905382124682
9 - loss : 494.6740031508518
10 - loss : 494.45965338940766
11 - loss : 494.24742961139583
12 - loss : 494.03727415771306
13 - loss : 493.82913098054036
14 - loss : 493.6229455982865
15 - loss : 493.41866505179297
16 - loss : 493.216237861762
17 - loss : 493.01561398737596
18 - loss : 492.81674478607323
19 - loss : 492.61958297445
20 - loss : 492.42408259025336
21 - loss : 492.2301989554401
22 - loss : 492.03788864026535
23 - loss : 491.84710942837734
24 - loss : 491.6578202828886
25 - loss : 491.4699813133939
26 - loss : 491.2835537439126
27 - loss : 491.0984998817258
28 - loss : 490.9147830870862
29 - loss : 490.7323677437743
30 - loss : 490.5512192304778
31 - loss : 490.3713038929714
32 - loss : 490.1925890170752
33 - loss